<a href="https://colab.research.google.com/github/DefinitelyKev/nlp_bert_ml_recipe_models/blob/main/whole_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**
---



In [ ]:
!pip install optimum[onnxruntime] onnx

In [ ]:
import os
from torch.nn.functional import softmax
import spacy
import torch.nn as nn
import torch
import pickle
import re
import requests
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import DistilBertConfig
import onnx
import onnxruntime as ort

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Model Path**
---



In [ ]:
ingredient_parsing_model_path = '/content/drive/MyDrive/ingredient_parsing_model'

segmentation_model_path = '/content/drive/MyDrive/segmentation_model/segmentation_model_v1'
segmentation_model_path_onnx = '/content/drive/MyDrive/segmentation_model/segmentation_model_v1.onnx'
segmentation_le_path = '/content/drive/MyDrive/segmentation_model/segmentation_model_label_encoder_v1/le.pkl'

cook_time_model_path = '/content/drive/MyDrive/cook_time_model/cook_time_model_v1'
cook_time_model_path_onnx = '/content/drive/MyDrive/cook_time_model/cook_time_model_v1.onnx'
cook_time_scaler_path = '/content/drive/MyDrive/cook_time_model/cook_time_scaler_v1/scaler.pkl'

tags_model_path = '/content/drive/MyDrive/tagging_model_tags/tags_model_v1'
tags_model_path_onnx = '/content/drive/MyDrive/tagging_model_tags/tags_model_v1.onnx'
tags_mlb_path = '/content/drive/MyDrive/tagging_model_tags/tags_mlb_v1/mlb.pkl'

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", token=access_token)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# **Model Definitions**
---



In [ ]:
# class AttentionLayer(nn.Module):
#     def __init__(self, in_features):
#         super(AttentionLayer, self).__init__()
#         self.attention = nn.Sequential(
#             nn.Linear(in_features, in_features),
#             nn.Tanh(),
#             nn.Linear(in_features, 1),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         weights = self.attention(x)
#         weighted = torch.mul(x, weights.expand_as(x))
#         return weighted.sum(1)

In [ ]:
# # Model Definition
# class DistilBertRegressor(nn.Module):
#     def __init__(self):
#         super(DistilBertRegressor, self).__init__()
#         self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased", token=access_token)
#         self.config = self.distilbert.config
#         self.attention = AttentionLayer(self.distilbert.config.dim)
#         self.regressor = nn.Sequential(
#             nn.Dropout(0.2),
#             nn.Linear(self.distilbert.config.dim, 1024),  # Increased input layer size
#             nn.LayerNorm(1024),
#             nn.GELU(),
#             nn.Dropout(0.1),
#             nn.Linear(1024, 512),
#             nn.LayerNorm(512),
#             nn.GELU(),
#             nn.Linear(512, 256),
#             nn.LayerNorm(256),
#             nn.GELU(),
#             nn.Linear(256, 128),
#             nn.LayerNorm(128),
#             nn.GELU(),
#             nn.Linear(128, 1)  # Output layer for two targets (cook_time, prep_time)
#         )
#         # Enhanced residual path with layer normalization
#         self.residual = nn.Sequential(
#             nn.Linear(self.distilbert.config.dim, 512),
#             nn.LayerNorm(512),
#             nn.GELU(),
#             nn.Linear(512, 1)
#         )

#     def forward(self, input_ids, attention_mask):
#         distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_state = distilbert_output[0][:, 0]  # Extract the [CLS] token's hidden state
#         attention_output = self.attention(hidden_state.unsqueeze(1)).squeeze(1)
#         regression_output = self.regressor(attention_output)
#         residual_output = self.residual(hidden_state)

#         # Combine outputs from the main regressor and the residual path
#         combined_output = regression_output + residual_output
#         return combined_output

In [ ]:
# # Model Definition
# class DistilBertForMultiLabelClassification(torch.nn.Module):
#     def __init__(self, num_labels=120, dropout_rate=0.3):
#         super().__init__()
#         self.num_labels = num_labels
#         self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased", token=access_token)
#         self.config = self.distilbert.config

#         # Expanded complex structure
#         self.dropout = nn.Dropout(dropout_rate)
#         self.dense1 = nn.Linear(self.distilbert.config.dim, self.distilbert.config.dim * 2)
#         self.activation1 = nn.GELU()
#         self.norm1 = nn.LayerNorm(self.distilbert.config.dim * 2)
#         self.dense2 = nn.Linear(self.distilbert.config.dim * 2, self.distilbert.config.dim * 2)
#         self.activation2 = nn.GELU()
#         self.norm2 = nn.LayerNorm(self.distilbert.config.dim * 2)
#         self.dense3 = nn.Linear(self.distilbert.config.dim * 2, self.distilbert.config.dim)  # bottleneck layer
#         self.activation3 = nn.GELU()
#         self.norm3 = nn.LayerNorm(self.distilbert.config.dim)

#         # Adding residual connections
#         self.residual1 = nn.Linear(self.distilbert.config.dim, self.distilbert.config.dim * 2)
#         self.residual2 = nn.Linear(self.distilbert.config.dim * 2, self.distilbert.config.dim * 2)

#         # Final classifier layer
#         self.classifier = nn.Linear(self.distilbert.config.dim, num_labels)

#     def forward(self, input_ids, attention_mask):
#         distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_state = distilbert_output[0]  # (bs, seq_length, dim)
#         pooled_output = hidden_state[:, 0]  # use the first token, typically [CLS]

#         # Passing through the complex network layers with residual connections
#         x = self.dropout(pooled_output)
#         x = self.dense1(x) + self.residual1(pooled_output)
#         x = self.activation1(x)
#         x = self.norm1(x)

#         x = self.dense2(x) + self.residual2(x)
#         x = self.activation2(x)
#         x = self.norm2(x)

#         x = self.dense3(x)
#         x = self.activation3(x)
#         x = self.norm3(x)

#         # Classifier
#         logits = self.classifier(x)
#         return logits

In [ ]:
# DistilBertSequenceModel = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4, token=access_token)

# **Loading Models**
---



In [ ]:
# def load_model_scaler(model_path, scaler_path, model, device):
#     model_state_dict = torch.load(os.path.join(model_path, 'model_state.bin'), map_location=device)
#     model.load_state_dict(model_state_dict)
#     model.to(device)
#     model.eval()

#     if "mlb" in scaler_path:
#         with open(os.path.join(scaler_path, 'mlb.pkl'), 'rb') as mlb_file:
#             scaler = pickle.load(mlb_file)
#     elif "label_encoder" in scaler_path:
#         with open(os.path.join(scaler_path, 'le.pkl'), 'rb') as label_encoder_file:
#             scaler = pickle.load(label_encoder_file)
#     else:
#       with open(os.path.join(scaler_path, 'scaler.pkl'), 'rb') as scaler_file:
#           scaler = pickle.load(scaler_file)

#     return model, scaler

# # Load model and scaler
# cook_time_model, cook_time_scaler = load_model_scaler(cook_time_model_path, cook_time_scaler_path, DistilBertRegressor(), device)
# tags_model, tags_mlb = load_model_scaler(tags_model_path, tags_mlb_path, DistilBertForMultiLabelClassification(), device)
# segmentation_model, segmentation_le = load_model_scaler(segmentation_model_path, segmentation_le_path, DistilBertSequenceModel, device)

In [ ]:
def load_scaler(scaler_path):
    with open(scaler_path, "rb") as scaler_file:
        scaler = pickle.load(scaler_file)
    return scaler

# Load model and scaler
cook_time_scaler = load_scaler(cook_time_scaler_path)
tags_mlb = load_scaler(tags_mlb_path)
segmentation_le = load_scaler(segmentation_le_path)

In [ ]:
# # Define dummy inputs
# def get_dummy_inputs(max_length=512):
#     input_ids = torch.randint(0, 100, (1, max_length)).to(device)
#     attention_mask = torch.ones_like(input_ids).to(device)
#     return input_ids, attention_mask


# # Convert model function
# def convert_to_onnx(model, model_name, dummy_inputs, output_path):
#     model.to(device)
#     model.eval()

#     input_names = ["input_ids", "attention_mask"]
#     output_names = ["output"]
#     dynamic_axes = {
#         "input_ids": {0: "batch_size", 1: "sequence_length"},
#         "attention_mask": {0: "batch_size", 1: "sequence_length"},
#         "output": {0: "batch_size"}
#     }

#     torch.onnx.export(
#         model,
#         dummy_inputs,
#         output_path,
#         input_names=input_names,
#         output_names=output_names,
#         dynamic_axes=dynamic_axes,
#         opset_version=11
#     )
#     print(f"{model_name} converted to ONNX and saved at {output_path}")

# # Load and convert DistilBertRegressor
# dummy_inputs = get_dummy_inputs()
# convert_to_onnx(cook_time_model, "DistilBertRegressor", dummy_inputs, cook_time_model_path_onnx)

# # Load and convert DistilBertForMultiLabelClassification
# dummy_inputs = get_dummy_inputs()
# convert_to_onnx(tags_model, "DistilBertForMultiLabelClassification", dummy_inputs, tags_model_path_onnx)

# # Load and convert DistilBertSequenceModel
# convert_to_onnx(segmentation_model, "DistilBertSequenceModel", dummy_inputs, segmentation_model_path_onnx)

In [ ]:
# Load the ONNX model
onnx_cook_time_session = ort.InferenceSession(cook_time_model_path_onnx)
onnx_tags_session = ort.InferenceSession(tags_model_path_onnx)
onnx_segmentation_session = ort.InferenceSession(segmentation_model_path_onnx)

# **Making Predictions**
---



In [ ]:
def encode_texts(data, tokenizer):
    if not isinstance(data, list):
        data = [data]

    encoding = tokenizer(
        data,
        max_length=512,
        padding="longest",
        truncation=True,
        return_tensors='pt'
    )
    return encoding['input_ids'], encoding['attention_mask']

In [ ]:
def preprocess_input_text(data, tokenizer, model_name):
    if model_name == "cook_time":
        combined_text = data["name"] + " [SEP] " + " ".join(data["ingredients_units"]) + " [SEP] " + " ".join(data["steps"]) + " [SEP] " + " ".join(data["tags"])
    elif model_name == "tags":
        combined_text = data["name"] + " [SEP] " + " ".join(data["ingredients_units"]) + " [SEP] " + " ".join(data["steps"])

    return encode_texts(combined_text, tokenizer) if model_name != "segmentation" else encode_texts(data, tokenizer)

In [ ]:
def onnx_predict(session, input_ids, attention_mask):
    input_feed = {
        "input_ids": input_ids.cpu().numpy(),
        "attention_mask": attention_mask.cpu().numpy(),
    }
    outputs = session.run(None, input_feed)
    return outputs

In [ ]:
def predict_regression_and_classification(model, model_name, input_ids, attention_mask, device, scaler):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    if model_name == "cook_time":
        outputs = onnx_predict(onnx_cook_time_session, input_ids, attention_mask)
        predictions = outputs[0]
    elif model_name == "segmentation":
        outputs = onnx_predict(onnx_segmentation_session, input_ids, attention_mask)
        logits = torch.tensor(outputs[0])
        probs = softmax(logits, dim=1)
        predicted_class = probs.argmax(dim=1)
        predictions = predicted_class.numpy()
    else:
        outputs = onnx_predict(onnx_tags_session, input_ids, attention_mask)
        probs = torch.tensor(outputs[0])
        predictions = torch.sigmoid(probs)
        predictions = predictions > 0.42
        predictions = predictions.numpy()

    predictions = scaler.inverse_transform(predictions)
    return predictions

# **Nutrition API**
---



In [ ]:
def get_nutrition(data):
    APP_ID = "a8208be0"
    API_KEY = "71cfd0401d28ac067a1d8e1ad32ddf38"
    url = "https://api.edamam.com/api/nutrition-details"

    headers = {"Content-Type": "application/json"}
    recipe = {
        "title": data["name"],
        "ingr": data["ingredients_units"],
    }

    # Parameters including your credentials
    params = {"app_id": APP_ID, "app_key": API_KEY}
    response = requests.post(url, headers=headers, json=recipe, params=params)

    response_data = None
    total_weight = 0
    health_labels = []
    diet_labels = []
    nutrition = {
        "calories": "",
        "total_fat": "",
        "saturated_fat": "",
        "trans_fat": "",
        "carbohydrates": "",
        "fiber": "",
        "sugars": "",
        "protein": "",
        "cholesterol": "",
        "sodium": "",
    }

    if response.status_code == 200:
        response_data = response.json()
        nutrients = response_data['totalNutrients']
        nutrition['calories'] = f"{nutrients['ENERC_KCAL']['quantity']:.2f} {nutrients['ENERC_KCAL']['unit']}"
        nutrition['total_fat'] = f"{nutrients['FAT']['quantity']:.2f} {nutrients['FAT']['unit']}"
        nutrition['saturated_fat'] = f"{nutrients['FASAT']['quantity']:.2f} {nutrients['FASAT']['unit']}"
        nutrition['trans_fat'] = f"{nutrients['FATRN']['quantity']:.2f} {nutrients['FATRN']['unit']}"
        nutrition['carbohydrates'] = f"{nutrients['CHOCDF']['quantity']:.2f} {nutrients['CHOCDF']['unit']}"
        nutrition['fiber'] = f"{nutrients['FIBTG']['quantity']:.2f} {nutrients['FIBTG']['unit']}"
        nutrition['sugars'] = f"{nutrients['SUGAR']['quantity']:.2f} {nutrients['SUGAR']['unit']}"
        nutrition['cholesterol'] = f"{nutrients['CHOLE']['quantity']:.2f} {nutrients['CHOLE']['unit']}"
        nutrition['sodium'] = f"{nutrients['NA']['quantity']:.2f} {nutrients['NA']['unit']}"
        nutrition['protein'] = f"{nutrients['PROCNT']['quantity']:.2f} {nutrients['PROCNT']['unit']}"

        health_labels = response_data['healthLabels']
        diet_labels = response_data['dietLabels']
        total_weight = response_data['totalWeight']

        health_labels = [health_label.lower() for health_label in health_labels]
        diet_labels = [diet_label.lower() for diet_label in diet_labels]
    else:
        print("Failed to fetch data:", response.status_code)


    return nutrition, health_labels, diet_labels, total_weight

# **Whole Model**
---



In [ ]:
def clean_text(text):
    if isinstance(text, list):
        return [clean_text_single(t) for t in text]
    else:
        return clean_text_single(text)

def clean_text_single(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'#\S+', '', text)  # Remove hashtags
    text = re.sub(r'^\d+\.\s+', '', text) # Remove numbers followed by a dot at the start of text
    text = re.sub(r'(\d+)(?=[^\d\s/\-.])', r'\1 ', text)  # Add space after numbers, unless followed by punctuation
    text = re.sub(r'[^\w\s/-]', '', text)  # Remove punctuation but keep /, -
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove emojis
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters like emojis
    text = re.sub(r'\s+', ' ', text).strip() # Normalize whitespace and strip leading/trailing spaces
    return text

def clean_list(strings):
    return [' '.join(s.split()) for s in strings if s.strip()]

In [ ]:
text = """CRISPY CAULIFLOWER AND GREEN TAHINI

Perfect for entertaining or as a healthy side dish

Ingredients:
Cauliflower
Olive oil
Dried garlic
Oregano
Onion powder
S&P

Green Tahini sauce:
1/2 cup Hulled tahini
1 bunch Parsley
1 bunch Mint
3 Garlic cloves
2 Lemons juiced
2 tablespoons honey

Method:
Heat oven to 200 degrees
Chop cauliflower into florets and roast with all non sauce ingredients for 45 mins
Use food processor to combine tahini sauce ingredients
Once cauliflower is crispy, take out of the oven and spoon tahini sauce on top
Serve as a side to any meal

This is my new dinner.
"""

lines = text.splitlines()
texts = [line for line in lines if line]
cleaned_texts = clean_text(texts)

In [ ]:
text = """
COOK ONCE EAT TWICE

~ CHICKEN KORMA TRAY-BAKE ~

Looking a quick and easy, warming, mid-week dinner? This one’s for you.

A great option for the whole family and perfect for leftovers.

The chicken can be marinated ahead of time and even frozen, ready for a later date.

Ingredients:

Serves 5

500g chicken thighs
2 tbs natural yoghurt
1 tbs curry paste (bought or homemade)

Curry Paste:
3 tbs cashew nuts
1 whole star anise
2 1/2 tsp cumin seeds
1 1/2 tbs coriander seeds
1/2 tsp turmeric
1 tsp garam masala
1/2 tsp chilli powder
2 tbs olive oil
3 cloves garlic
2 tbs grated ginger
2 tbs tomato paste
1 1/2 tbs dessicated coconut
3 tbs fresh coriander

1 onion, diced
1 tbs grated ginger
2 cloves garlic, crushed
1/2 cup of Korma curry paste
1/2 tsp ground cardamom
2 tbs tomato paste
400mL coconut milk
1 bunch Dutch carrots
1 medium head of broccoli, cut into florets
1-2 cups chopped spinach
Rice, to serve

Method:
1. Put chicken, yoghurt and 1 tbs of curry paste into a bowl. Mix to combine. Set aside for at least 20 mins to marinate.
2. Preheat the oven to 180C.
3. Heat an oven-proof tray/pan over medium heat and add a drizzle of olive oil and chicken. Cook for 1-2 mins each side to brown. Remove from pan.
4. Add onion, garlic and ginger and sauté for 3-4 minutes. Add curry paste, cardamom, tomato paste and coconut milk. Mix to combine and bring to a simmer.
5. Add chicken back in and carrots and put tray into the oven and cook for 20 minutes.
6. Chop broccoli and spinach. After 20 minutes, mix in the spinach and add the broccoli. Cook for a further 10 minutes.
7. Remove from the oven, add coriander and serve with rice or naan bread.

#korma #kormacurry #chickenkorma #indiancooking #dinner #weeknightdinner #quickdinner #onetraydinner #onepanmeal #mealideas #dinnerideas #quickandeasydinner #midweekmeal #traybake #cookonceeattwice #curry #winterdinners #whatspruecooking #dietitian #nutrition #nutritionist #wholefoods #recipeideas #mumof3 #family #familymealideas #homecook #wholesomefamilymeals #dietitianapproved #dietitiansofinstagram
"""

lines = text.splitlines()
texts = [line for line in lines if line]
cleaned_texts = clean_list(clean_text(texts))


In [ ]:
text = """
~ ANZAC BISCUITS ~

Tomorrow is ANZAC Day.

Each year around Anzac Day I’ll make a batch or two of Anzac biscuits and think ‘why don’t I make these all year round’ 🤔.

They’re so delicious and they keep for ages in an airtight container, which is how they were designed back in the day to be able to be shipped to the soldiers fighting to defend our country.

Lest we forget.

Ingredients:
1 1/2 cups rolled oats
1 cup plain flour
1/2 cup raw sugar
3/4 cup shredded coconut
125g butter
1/3 cup golden syrup
1 tsp bicarb soda
2 tbs boiling water

Method:
1. Preheat the oven to 16oºC and line a baking tray with baking paper.
2. Combine oats, flour, sugar and coconut h into a bowl and mix to combine.
3. Place the butter and golden syrup into and small saucepan and heat over medium heat until the mixture starts to bubble slightly.
4. Place the bicarb soda and water into a small bowl, mix quickly then pour into the saucepan with the golden syrup. 5. Mix and quickly pour into the bowl with the dry ingredients and mix well to combine.
6. Roll spoonfuls of the mix into balls and place onto the tray. Gently press down with a fork and repeat with the remaining mixture.
7. Bake for ~12-15 minutes or until golden brown.

#anzacday #anzacbiscuits #anzacbiscuitrecipe #lestweforget #whatspruecooking #dietitian #nutrition #nutritionist #delicious #kidssnacks #snacks #lunchboxsnacks #family #familysnacks #wholesomefamilymeals #wholesomesnacks #deliciousbiscuits #atthegoingdownofthesun #anzac #anzacspirit
"""

lines = text.splitlines()
texts = [line for line in lines if line]
cleaned_texts = clean_list(clean_text(texts))

In [ ]:
cleaned_texts

['crispy cauliflower and green tahini',
 'perfect for entertaining or as a healthy side dish',
 'ingredients',
 'cauliflower',
 'olive oil',
 'dried garlic',
 'oregano',
 'onion powder',
 'sp',
 'green tahini sauce',
 '1/2 cup hulled tahini',
 '1 bunch parsley',
 '1 bunch mint',
 '3 garlic cloves',
 '2 lemons juiced',
 '2 tablespoons honey',
 'method',
 'heat oven to 200 degrees',
 'chop cauliflower into florets and roast with all non sauce ingredients for 45 mins',
 'use food processor to combine tahini sauce ingredients',
 'once cauliflower is crispy take out of the oven and spoon tahini sauce on top',
 'serve as a side to any meal',
 'this is my new dinner']

In [ ]:
import json
ingredients_model = spacy.load("/content/drive/MyDrive/ingredient_parsing_model")

In [ ]:
def predict_with_model(texts, model_name, model, scaler):
    """ Predicts tags for given recipe data. """
    input_ids, attention_mask = preprocess_input_text(texts, tokenizer, model_name)
    predictions = predict_regression_and_classification(model, model_name, input_ids, attention_mask, device, scaler)

    if model_name == "tags":
        predictions = list(predictions[0])
    elif model_name == "segmentation":
        predictions = [(text, label) for text, label in zip(texts, predictions)]
    else:
        predictions = 5 * round(predictions[0][0] / 5)
    return predictions


In [ ]:
def parse_ingredients(text, model):
    """ Parses ingredients from text using a Spacy NER model. """
    doc = model(text)
    return [{"text": ent.text, "label": ent.label_} for ent in doc.ents]

def predict_ingredients(raw_ingredients, ingredients_model):
    """ Predicts ingredients details and extracts unique ingredients set. """
    ingredient_details, NER_ingredients = {}, set()

    for name, ingredients_list in raw_ingredients.items():
        ingredient_details[name] = {}
        for ingredient_text in ingredients_list:
            parsed_ingredients = parse_ingredients(ingredient_text, ingredients_model)
            food_name, serving_size, unit, modifier, colour, physical_quality = 'Unknown', 'unspecified', '', '', '', ''

            for item in parsed_ingredients:
                if item['label'] == 'QUANTITY':
                    serving_size = item['text']
                elif item['label'] == 'UNIT':
                    unit = item['text']
                elif item['label'] in ['PROCESS']:
                    modifier = item['text']
                elif item['label'] == 'COLOR':
                    colour = item['text']
                elif item['label'] == 'PHYSICAL_QUALITY':
                    physical_quality = item['text']
                elif item['label'] == 'FOOD':
                    food_name = item['text']

            food_name = " ".join(f"{colour} {physical_quality} {food_name}".split()) if food_name != 'Unknown' else ingredient_text
            serving = " ".join(f"{serving_size} {unit} {modifier}".split())
            NER_ingredients.add(food_name)
            ingredient_details[name][food_name] = serving

    return ingredient_details, NER_ingredients

def categorize_text(data):
    """ Categorizes text into recipes or steps based on their labels. """
    recipes, steps = {}, []
    current_name, first_name = None, None

    for text, label in data:
        if text in ['method', 'steps', 'ingredients', 'ingredient', 'method:']:
            continue

        if label == 'name':
            current_name = text
            recipes[current_name] = []
            if not first_name:
                first_name = text
        elif label == 'ingredients' and current_name:
            recipes[current_name].append(text)
            if first_name and first_name != current_name:
                recipes[first_name].append(text)
        elif label == 'steps':
            steps.append(text)

    return recipes, steps

def format_steps(raw_steps):
    """ Formats the raw steps into a numbered dictionary. """
    return {idx + 1: step for idx, step in enumerate(raw_steps)}

def get_servings(full_servings, tags):
    """ Determines number of servings and servings size based on tags. """
    servings_size = 200
    servings_size_map = {
        "desserts": 135,
        "cookies-and-brownies": 135,
        "meat": 400,
        "poultry": 400,
        "vegetarian": 300,
        "vegetables": 300
    }
    for tag in tags:
        if tag in servings_size_map:
            servings_size = servings_size_map[tag]

    servings = full_servings // servings_size
    return int(servings), servings_size

def get_difficulty(number_of_ingredients, number_of_steps):
    """ Determines the cooking difficulty based on ingredients and steps. """
    total_items = number_of_ingredients + number_of_steps
    if total_items <= 20:
        return "easy"
    elif total_items <= 40:
        return "medium"
    else:
        return "hard"

# segmentation model
segmented_data = predict_with_model(cleaned_texts, "segmentation", onnx_segmentation_session, segmentation_le)
raw_ingredients, raw_steps = categorize_text(segmented_data)

recipe_name = (next(iter(raw_ingredients)))
sub_names = [name for name in raw_ingredients.keys() if name != recipe_name]

nutrition_recipe_data = {
    "name": recipe_name,
    "ingredients_units": raw_ingredients[recipe_name],
}

tags_recipe_data = nutrition_recipe_data.copy()
tags_recipe_data["steps"] = raw_steps

# tag model
tags = predict_with_model(tags_recipe_data, "tags", onnx_tags_session, tags_mlb)

cook_time_recipe_data = tags_recipe_data.copy()
cook_time_recipe_data["tags"] = tags

# cook time model
cook_time = predict_with_model(cook_time_recipe_data, "cook_time", onnx_cook_time_session, cook_time_scaler)

# ingredient parsing model
ingredients, NER_ingredients = predict_ingredients(raw_ingredients, ingredients_model)

# nutrition api
nutrition, health_labels, diet_labels, full_servings = get_nutrition(nutrition_recipe_data)

servings, serving_size = get_servings(full_servings, tags)
main_ingredients = ingredients[recipe_name]
sub_ingredients = {name: ingredients[name] for name in sub_names}
steps = format_steps(raw_steps)
difficulty = get_difficulty(len(main_ingredients), len(steps))

final_recipe = {
    "name": recipe_name,
    "tags": {
        "difficulty": difficulty,
        "servings": servings,
        "serving_size_grams": serving_size,
        "cooking_time_minutes": cook_time,
        "tags": tags
    },
    "ingredients": main_ingredients,
    "steps": steps,
    "nutrition": nutrition,
    "health_labels": health_labels,
    "diet_labels": diet_labels,
    "NER": list(NER_ingredients),
    "sub_recipes": []
}

if sub_names:
    for recipe_name in sub_names:
        final_recipe["sub_recipes"].append(
            {
                "name": recipe_name,
                "ingredients": sub_ingredients[recipe_name]
            }
        )

print(json.dumps(final_recipe, indent=4))


Failed to fetch data: 555
{
    "name": "crispy cauliflower and green tahini",
    "tags": {
        "difficulty": "easy",
        "servings": 0,
        "serving_size_grams": 300,
        "cooking_time_minutes": 50,
        "tags": [
            "30-minutes-or-less",
            "60-minutes-or-less",
            "main-dish",
            "vegetables",
            "vegetarian"
        ]
    },
    "ingredients": {
        "cauliflower": "unspecified",
        "olive oil": "unspecified",
        "dried garlic": "unspecified",
        "oregano": "unspecified",
        "onion powder": "unspecified",
        "hulled tahini": "1/2 cup",
        "parsley": "1 bunch",
        "mint": "1 bunch",
        "garlic": "3 cloves",
        "lemons": "2 juiced",
        "honey": "2 tablespoons"
    },
    "steps": {
        "1": "heat oven to 200 degrees",
        "2": "chop cauliflower into florets and roast with all non sauce ingredients for 45 mins",
        "3": "use food processor to combine tahin

In [ ]:
# import spacy
# from flask import Flask, request, jsonify
# import re

# app = Flask(__name__)
# model = spacy.load("./models/ingredient_parsing_model_v1")



# @app.route('/predict', methods=['POST'])
# def predict():
#     if not request.is_json:
#         return jsonify({"error": "Request must be JSON"}), 400

#     data = request.get_json()
#     if 'text' not in data:
#         return jsonify({"error": "Missing 'text' field in JSON data"}), 400

#     text = data['text']
#     if not isinstance(text, str) or text.strip() == "":
#         return jsonify({"error": "The 'text' field must be a non-empty string"}), 400

#     text = re.sub(r'\s+', ' ', text.strip())
#     lines = text.splitlines()

#     texts = [line for line in lines if line]
#     cleaned_texts = clean_text(texts)

#     segmented_data = predict_segmentation(cleaned_texts)
#     raw_ingredients, raw_steps = categorize_text(segmented_data)

#     recipe_name = (next(iter(raw_ingredients)))

#     tags_recipe_data = {
#         "name": recipe_name,
#         "ingredients_units": raw_ingredients[recipe_name],
#         "steps": raw_steps
#     }

#     tags = predict_tags(tags_recipe_data)
#     ingredients = predict_ingredients(raw_ingredients)

#     # Nutrition, Tagging, Cooking time
#     def process_lines(lines):
#         # Your model processing code would go here
#         # This function should take the lines, apply model predictions and return results
#         # Example:
#         return {"lines_processed": len(lines), "details": processed_lines}

#     # Get model predictions or processing results
#     try:
#         result = process_lines(lines)
#     except Exception as e:
#         return jsonify({"error": f"Error processing the text: {str(e)}"}), 500

#     return jsonify(result)

# if __name__ == '__main__':
#     app.run(debug=True, host='0.0.0.0')


In [ ]:
# Convert tags to a binary matrix
mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(df['tags'])
tags_df = pd.DataFrame(tags_encoded, columns=mlb.classes_)

# Create a feature matrix X and labels matrix Y
X = np.random.rand(tags_encoded.shape[0], 10)  # Random feature data
Y = tags_df.values

# Initialize the MultiSmote object
smote = MultiSmote()

# Apply ML-SMOTE
X_resampled, Y_resampled = smote.multi_smote(X, Y)

# Convert the resampled labels back to DataFrame to check the new distribution
Y_resampled_df = pd.DataFrame(Y_resampled, columns=mlb.classes_)
new_tag_counts = Y_resampled_df.sum(axis=0)

print(new_tag_counts)